# IATA

### Goal:
Find and merge the following information for each international travel route:
1. Origin country: ISO 3 code, country centroid coordiantes.
* Destination county: county FIPS, county centroid coordinates.
* Existing data: travel volume for each route.

### Challenges:
1. Clear country names and apply the same unique id - ISO 3 code. 
    - Requires: original IATA spreadsheet (`IATA_country_USairport_county_07_17.csv` or `df_iata`) + country name - ISO 3 code conversion spreadsheet (`origin_XY.csv` or `df_origin`).
* Find the coordiantes for all airports with their IATA code.
    - From Lauren: IATA code and coordiantes for each US airport.

### Update info:
* Initial draft compiled on 4/30/2019.
* A complete update on 6/13.

In [1]:
import pandas as pd

### Import original IATA travel data

In [24]:
folder = r'C:\Users\Ensheng\Desktop\mapping\IATA\\'
in_table = folder + 'IATA_country_USairport_county_07_17.csv'
df_iata = pd.read_csv(in_table)
print(len(df_iata))
df_iata.head(5)

370975


,year,originCountryId,originCountryName,destinationAirportName,destinationAirportCode,destinationAirportId,destination_state,destination_county,destination_countyDetailed,paxVolume
0,2007,73,United Kingdom,Naval Outlying Field Barin,NHX,3225,Alabama,Baldwin,Baldwin County,3
1,2007,101,Italy,Muscle Shoals Regional Airport,MSL,3210,Alabama,Colbert,Colbert County,1
2,2007,102,Jamaica,Muscle Shoals Regional Airport,MSL,3210,Alabama,Colbert,Colbert County,75
3,2007,103,Jordan,Muscle Shoals Regional Airport,MSL,3210,Alabama,Colbert,Colbert County,2
4,2007,104,Japan,Muscle Shoals Regional Airport,MSL,3210,Alabama,Colbert,Colbert County,2


In [25]:
# find airports number for each year
print("Info: Number of airports in IATA data set: " + str(df_iata['destinationAirportCode'].nunique()))
df_count = df_iata[['year','destinationAirportCode']]
df_count = df_count.drop_duplicates()
print("Info: Number of airports in each year:")
print(df_count['year'].value_counts().sort_index())

Info: Number of airports in IATA data set: 679
Info: Number of airports in each year:
2007    555
2008    500
2009    448
2010    445
2011    445
2012    435
2013    433
2014    435
2015    431
2016    421
2017    478
Name: year, dtype: int64


In [26]:
# slicing (optional)
# year = 2017
# df_iata = df_iata.loc[df_iata['year'] == year]
# print(len(df_iata))
# df_iata.head(5)

### Challenge 1: Clear the unique id - country name

In [27]:
df_iata['originRegion'] = df_iata['originCountryName'].replace({
    'Ã…land': 'Aland',
    'Antigua-Barbuda': 'Antigua and Barbuda',
    'Bahamas': 'The Bahamas',
    'Bonaire, Saint Eustatius and Saba': 'Caribbean Netherlands',
    'Bosnia-Herzegovina': 'Bosnia and Herzegovina',
    'Brunei Darussalam': 'Brunei',
    'Central African Rep': 'Central African Republic',
    #'Christmas Island': '',
    'Congo (Brazzaville)': 'Republic of Congo',
    'Congo (Kinshasa)': 'Democratic Republic of the Congo',
    'Cote D\'Ivoire': 'Ivory Coast',
    'Curacao': 'Curacao',
    'Dominican Rep': 'Dominican Republic',
    #'French Guiana': '',
    #'Gibraltar': '',
    #'Guadeloupe': '',
    'Guinea-Bissau': 'Guinea Bissau',
    'Hong Kong (SAR)': 'Hong Kong S.A.R.',
    'Korea (North)': 'North Korea',
    'Korea (South)': 'South Korea',
    'Macao (SAR)': 'Macao S.A.R',
    #'Martinique': '',
    #'Mayotte': '',
    'Micronesia': 'Federated States of Micronesia',
    'Netherlands Antilles (obsolete)': 'Netherlands Antilles',
    'North Mariana Isl': 'Northern Mariana Islands',
    #'Reunion': '',
    'Sao Tome-Principe': 'Sao Tome and Principe',
    'Serbia': 'Republic of Serbia',
    'St Barthelemy': 'Saint Barthelemy',
    'St Kitts-Nevis': 'Saint Kitts and Nevis',
    'St Lucia': 'Saint Lucia',
    'St Maarten (Dutch Part)': 'Sint Maarten',
    'St Martin': 'Saint Martin',
    'St Pierre-Miquelon': 'Saint Pierre and Miquelon',
    'St Vincent-Grenad': 'Saint Vincent and the Grenadines',
    #'Svalbard': '',
    #'Timor-leste': '',
    'Trinidad-Tobago': 'Trinidad and Tobago',
    'Turks-Caicos': 'Turks and Caicos Islands',
    #'Tuvalu': '',
    'Viet Nam': 'Vietnam',
    'Virgin Islands (GB)': 'British Virgin Islands',
    'Virgin Islands (US)': 'United States Virgin Islands',
    'Wallis-Futuna Isl': 'Wallis and Futuna'})
print(len(df_iata))
df_iata.head(5)

370975


,year,originCountryId,originCountryName,destinationAirportName,destinationAirportCode,destinationAirportId,destination_state,destination_county,destination_countyDetailed,paxVolume,originRegion
0,2007,73,United Kingdom,Naval Outlying Field Barin,NHX,3225,Alabama,Baldwin,Baldwin County,3,United Kingdom
1,2007,101,Italy,Muscle Shoals Regional Airport,MSL,3210,Alabama,Colbert,Colbert County,1,Italy
2,2007,102,Jamaica,Muscle Shoals Regional Airport,MSL,3210,Alabama,Colbert,Colbert County,75,Jamaica
3,2007,103,Jordan,Muscle Shoals Regional Airport,MSL,3210,Alabama,Colbert,Colbert County,2,Jordan
4,2007,104,Japan,Muscle Shoals Regional Airport,MSL,3210,Alabama,Colbert,Colbert County,2,Japan


### Goal 1: Find all country ISO 3 code and centroids

In [28]:
# import country ISO 3 code conversion spreadsheet
in_table = folder + 'origin_XY.csv'
df_origin = pd.read_csv(in_table)
print(len(df_origin))
df_origin.head(5)

253


,Country,ISO,Origin_X,Origin_Y,region_un,subregion
0,Afghanistan,AFG,66.004734,33.835231,Asia,Southern Asia
1,Aland,ALD,19.953288,60.214887,Europe,Northern Europe
2,Albania,ALB,20.049834,41.142450,Europe,Southern Europe
3,Algeria,DZA,2.617323,28.158938,Africa,Northern Africa
4,American Samoa,ASM,-170.718026,-14.304460,Oceania,Polynesia


In [29]:
result = pd.merge(df_iata, df_origin, how='left', left_on='originRegion',right_on='Country')
print(len(result))
result.head(5)

370975


,year,originCountryId,originCountryName,destinationAirportName,destinationAirportCode,destinationAirportId,destination_state,destination_county,destination_countyDetailed,paxVolume,originRegion,Country,ISO,Origin_X,Origin_Y,region_un,subregion
0,2007,73,United Kingdom,Naval Outlying Field Barin,NHX,3225,Alabama,Baldwin,Baldwin County,3,United Kingdom,United Kingdom,GBR,-2.865632,54.123872,Europe,Northern Europe
1,2007,101,Italy,Muscle Shoals Regional Airport,MSL,3210,Alabama,Colbert,Colbert County,1,Italy,Italy,ITA,12.070013,42.796626,Europe,Southern Europe
2,2007,102,Jamaica,Muscle Shoals Regional Airport,MSL,3210,Alabama,Colbert,Colbert County,75,Jamaica,Jamaica,JAM,-77.314826,18.156949,Americas,Caribbean
3,2007,103,Jordan,Muscle Shoals Regional Airport,MSL,3210,Alabama,Colbert,Colbert County,2,Jordan,Jordan,JOR,36.771361,31.245791,Asia,Western Asia
4,2007,104,Japan,Muscle Shoals Regional Airport,MSL,3210,Alabama,Colbert,Colbert County,2,Japan,Japan,JPN,138.030896,37.592301,Asia,Eastern Asia


In [30]:
# make sure no country missing ISO 3
result.isnull().sum()

year                          0
originCountryId               0
originCountryName             0
destinationAirportName        0
destinationAirportCode        0
destinationAirportId          0
destination_state             0
destination_county            0
destination_countyDetailed    0
paxVolume                     0
originRegion                  0
Country                       0
ISO                           0
Origin_X                      0
Origin_Y                      0
region_un                     0
subregion                     0
dtype: int64

### Challenge 2: Find county centroid cooridantes for all `df_iata` airports

In [31]:
# import airport data getting from the internet
in_table = folder + 'IATA_airports.xlsx' # avoid ''utf-8' error (csv -> xlsx)
df_airports = pd.read_excel(in_table)
print(len(df_airports))
df_airports.head(5)

11846


,Code,Name,City,State,State Name,Latitude,Longitude,Country,Country Name,Global Region
0,AAA,Anaa,Anaa,,,-17.35,-145.50,PF,French Polynesia,Australasia
1,AAB,Arrabury,Arrabury,QL,Queensland,-26.70,141.04,AU,Australia,Australasia
2,AAC,El Arish International,El Arish,,,31.08,33.83,EG,Egypt,Africa
3,AAD,Ad-Dabbah Airport,Ad-Dabbah,,,18.05,30.95,SD,Sudan,Africa
4,AAE,Rabah Bitat,Annaba,,,36.82,7.81,DZ,Algeria,Africa


In [32]:
# find airports in df_iata but not in df_airports
# ref: https://stackoverflow.com/questions/19960077/how-to-implement-in-and-not-in-for-pandas-dataframe
missing_set = set(df_iata['destinationAirportCode'].unique()) - set(df_airports['Code'].unique())
print("Info: total airports in IATA data set: " + str(df_iata['destinationAirportCode'].nunique()))
missing_cnt = len(missing_set)
print("Warning: we are missing " + str(missing_cnt) + " airports.")

df_iata_selected = df_iata.groupby(['destinationAirportCode'])['paxVolume'].sum().reset_index()
df_iata_selected = df_iata_selected.loc[df_iata_selected['paxVolume'] < 5]
print("Info: total airports in IATA dataset having less than 5 paxVolume: " + str(df_iata_selected['destinationAirportCode'].nunique()))

Info: total airports in IATA data set: 679
Info: total airports in IATA dataset having less than 5 paxVolume: 112


In [33]:
if missing_cnt > 0:    
    # output: all missing airports
    df_missing = df_iata[df_iata.destinationAirportCode.isin(missing_set)].sort_values(by='destinationAirportCode')
    df_missing = df_missing.groupby(['destinationAirportCode','destinationAirportName','destination_state','destination_county','destination_countyDetailed'])['paxVolume'].sum().reset_index()
    output_csv = folder + 'missing_airports.csv'
    df_missing.to_csv(output_csv,index = False,encoding='utf-8')
    print("Info: " + str(len(df_missing)) + " missing airports table exported.")
    print(df_missing.head(5))

else:
    # output: all IATA required airports (count should be 679)
    required_ariport_set = set(df_iata['destinationAirportCode'].unique())
    df_required_ariport = df_airports[df_airports.Code.isin(required_ariport_set)]
    output_csv = folder + 'required_ariports.csv'
    df_required_ariport.to_csv(output_csv,index = False,encoding='utf-8')
    print("Info: " + str(len(df_required_ariport)) + " required airports table exported.")
    print(df_required_ariport.head(5))

Info: 679 required airports table exported.
   Code                         Name         City State    State Name  \
30  ABE  Lehigh Valley International    Allentown    PA  Pennsylvania   
34  ABI     Abilene Regional Airport      Abilene    TX         Texas   
42  ABQ        International Sunport  Albuquerque    NM    New Mexico   
43  ABR            Aberdeen Regional     Aberdeen    SD  South Dakota   
50  ABY       Southwest Georgia Rgnl       Albany    GA       Georgia   

    Latitude  Longitude Country   Country Name  Global Region  
30     40.65     -75.44      US  United States  North America  
34     32.41     -99.68      US  United States  North America  
42     35.04    -106.61      US  United States  North America  
43     45.45     -98.43      US  United States  North America  
50     31.53     -84.20      US  United States  North America  


#### Preparation in ArcMap

1. Erase to find all out-of-polgyon airports (the projection matters!).
* Adjust locations or remove points for out-of-polgyon airports.
* Intersect all 676 airports with us_county_3220_centroids_wgs84 to spatially join county X and county Y to each airport.
* Export the output as `airports_countyXY.csv`.

In [34]:
in_table = folder + 'airports_countyXY.csv'
ddf = pd.read_csv(in_table)
print(len(ddf))
ddf.head(3)

676


,OBJECTID,FID_IATA_airports_679_wgs84,Code,Name,City,State,State_Name,Latitude,Longitude,Country,...,GEOID,NAME_1,LSAD,ALAND,AWATER,FIPS,State_1,ST,County_X,County_Y
0,1,351,KOA,Keahole,Kona/Hawaii,HI,Hawaii,19.65,-156.01,US,...,15001,Hawaii,6,1.043364e+10,2.739470e+09,15001,Hawaii,HI,-155.521017,19.601212
1,2,327,ITO,Hilo International,Hilo/Hawaii,HI,Hawaii,19.72,-155.06,US,...,15001,Hawaii,6,1.043364e+10,2.739470e+09,15001,Hawaii,HI,-155.521017,19.601212
2,3,449,MUE,Waimea-Kohala,Kamuela,HI,Hawaii,20.01,-155.67,US,...,15001,Hawaii,6,1.043364e+10,2.739470e+09,15001,Hawaii,HI,-155.521017,19.601212


In [35]:
result = pd.merge(result, ddf, how='left', left_on='destinationAirportCode',right_on='Code')
print(len(result))
result.head(5)

370975


,year,originCountryId,originCountryName,destinationAirportName,destinationAirportCode,destinationAirportId,destination_state,destination_county,destination_countyDetailed,paxVolume,...,GEOID,NAME_1,LSAD,ALAND,AWATER,FIPS,State_1,ST,County_X,County_Y
0,2007,73,United Kingdom,Naval Outlying Field Barin,NHX,3225,Alabama,Baldwin,Baldwin County,3,...,1003.0,Baldwin,6.0,4.117606e+09,1.133109e+09,1003.0,Alabama,AL,-87.722071,30.727750
1,2007,101,Italy,Muscle Shoals Regional Airport,MSL,3210,Alabama,Colbert,Colbert County,1,...,1033.0,Colbert,6.0,1.534875e+09,8.002714e+07,1033.0,Alabama,AL,-87.801685,34.698475
2,2007,102,Jamaica,Muscle Shoals Regional Airport,MSL,3210,Alabama,Colbert,Colbert County,75,...,1033.0,Colbert,6.0,1.534875e+09,8.002714e+07,1033.0,Alabama,AL,-87.801685,34.698475
3,2007,103,Jordan,Muscle Shoals Regional Airport,MSL,3210,Alabama,Colbert,Colbert County,2,...,1033.0,Colbert,6.0,1.534875e+09,8.002714e+07,1033.0,Alabama,AL,-87.801685,34.698475
4,2007,104,Japan,Muscle Shoals Regional Airport,MSL,3210,Alabama,Colbert,Colbert County,2,...,1033.0,Colbert,6.0,1.534875e+09,8.002714e+07,1033.0,Alabama,AL,-87.801685,34.698475


In [36]:
# list airports not in the airports_countyXY.csv (the same as changes in ArcMap)
df_null = result.loc[result['Code'].isnull()]
null_airports = df_null.groupby(['destinationAirportCode','destinationAirportName','destination_state','destination_county','destination_countyDetailed'])['paxVolume'].sum().reset_index()
output_csv = folder + 'null_airports.csv'
null_airports.to_csv(output_csv,index = False,encoding='utf-8')
null_airports

,destinationAirportCode,destinationAirportName,destination_state,destination_county,destination_countyDetailed,paxVolume
0,BAR,"""""",Alaska,Prince of Wales-Hyder,Prince of Wales-Hyder Census Area,1
1,KCK,"""""",Kansas,Wyandotte,Wyandotte County,4
2,MDO,Middleton Island Airport,Alaska,Valdez-Cordova,Valdez-Cordova Census Area,1


In [37]:
# select all not null rows
result = result.loc[result['Code'].notnull()]
result = result[['year',
                 'originCountryId',
                 'originRegion',
                 'ISO',
                 'Origin_X',
                 'Origin_Y',
                 'region_un',
                 'subregion',
                 'Code',
                 'State',
                 'STATEFP',
                 'COUNTYFP',
                 'FIPS',
                 'County_X',
                 'County_Y',
                 'paxVolume']]

In [42]:
# Find the country with the largest incoming travel volume to each county
result['Pax_Rank'] = result.groupby(['year','FIPS'])['paxVolume'].rank(ascending=False,method='dense')
result.head(5)

,year,originCountryId,originRegion,ISO,Origin_X,Origin_Y,region_un,subregion,Code,State,STATEFP,COUNTYFP,FIPS,County_X,County_Y,paxVolume,Pax_Rank,flowid
0,2007,73,United Kingdom,GBR,-2.865632,54.123872,Europe,Northern Europe,NHX,AL,1.0,3.0,1003.0,-87.722071,30.727750,3,1.0,1
1,2007,101,Italy,ITA,12.070013,42.796626,Europe,Southern Europe,MSL,AL,1.0,33.0,1033.0,-87.801685,34.698475,1,14.0,2
2,2007,102,Jamaica,JAM,-77.314826,18.156949,Americas,Caribbean,MSL,AL,1.0,33.0,1033.0,-87.801685,34.698475,75,2.0,3
3,2007,103,Jordan,JOR,36.771361,31.245791,Asia,Western Asia,MSL,AL,1.0,33.0,1033.0,-87.801685,34.698475,2,13.0,4
4,2007,104,Japan,JPN,138.030896,37.592301,Asia,Eastern Asia,MSL,AL,1.0,33.0,1033.0,-87.801685,34.698475,2,13.0,5


In [12]:
# reduce to the top 3 for each county
#result = result.loc[result['Pax_Rank'] <= 10]

In [41]:
# output as a csv table
result['flowid'] = result.index + 1
output_csv = folder + 'flow_XY.csv'
result.to_csv(output_csv,index = False,encoding='utf-8')

In [16]:
# slicing for test
# result['flowid'] = result.index + 1
# output_csv = folder + 'flow_XY_' +  str(year) + '.csv'
# result.to_csv(output_csv,index = False,encoding='utf-8')